In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
import scipy
import sys
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [4]:
audio_data = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/rock/rock.00029.wav'
data,sr = librosa.load(audio_data)
Audio(data, rate=sr)

In [5]:
df = pd.read_csv('/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv')
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [6]:
df = df.drop("filename", axis=1)
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [7]:
genre_list = df.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
y

array([0, 0, 0, ..., 9, 9, 9])

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(np.array(df.iloc[:, :-1], dtype=float))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [10]:
from keras.models import Sequential
model = Sequential()
model.add(layers.Dense(256, activation="relu", input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [11]:
out = model.fit(X_train, y_train, epochs = 100, batch_size = 128)

Epoch 1/100
59/59 [==============================] - 0s 2ms/step - loss: 1.4456 - accuracy: 0.4988
Epoch 2/100
59/59 [==============================] - 0s 2ms/step - loss: 0.8499 - accuracy: 0.7116
Epoch 3/100
59/59 [==============================] - 0s 2ms/step - loss: 0.6642 - accuracy: 0.7772
Epoch 4/100
59/59 [==============================] - 0s 2ms/step - loss: 0.5346 - accuracy: 0.8226
Epoch 5/100
59/59 [==============================] - 0s 3ms/step - loss: 0.4513 - accuracy: 0.8496
Epoch 6/100
59/59 [==============================] - 0s 3ms/step - loss: 0.3776 - accuracy: 0.8787
Epoch 7/100
59/59 [==============================] - 0s 2ms/step - loss: 0.3277 - accuracy: 0.8944
Epoch 8/100
59/59 [==============================] - 0s 3ms/step - loss: 0.2719 - accuracy: 0.9143
Epoch 9/100
59/59 [==============================] - 0s 3ms/step - loss: 0.2203 - accuracy: 0.9354
Epoch 10/100
59/59 [==============================] - 0s 3ms/step - loss: 0.1917 - accuracy: 0.9451
Epoch 11/

59/59 [==============================] - 0s 3ms/step - loss: 0.0125 - accuracy: 0.9975
Epoch 83/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0072 - accuracy: 0.9983
Epoch 84/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0050 - accuracy: 0.9988
Epoch 85/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0044 - accuracy: 0.9988
Epoch 86/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0044 - accuracy: 0.9987
Epoch 87/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0036 - accuracy: 0.9991
Epoch 88/100
59/59 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 0.9985
Epoch 89/100
59/59 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 0.9989
Epoch 90/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 0.9987
Epoch 91/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.9988
Epoch 92/100


In [1]:
model.evaluate(X_test, y_test, batch_size = 128)

NameError: name 'model' is not defined